## Data Engineering Nanodegree Capstone Project

#### Promoting Trade and Tourism in the US using I-94 Immigration Data

- The purpose of this project is to build a Spark data warehouse and corresponding data marts and dashboards that would enable the US National Tourism and Trade Office to promote trade and tourism in the United States.

- Advertising expenditure could be allocated to promoting the US in countries that send few tourists to the US.

- Trade policy could be focused on countries that send the most number of visitors on business visas to the US.

- Similarly, tourist attractions and natural wonders in states and cities with little tourism could be promoted.

- Universities and colleges could identify the countries that send the most number of students to the US, and actively try to recruit students from those countries to their institutions.

- Airlines could plan their routes and adjust their schedules based on arrivals data.

***

### 1. Gather and Analyze Data

- Load the data from CSV and JSON files into Spark data frames.
- Check the schema and a sampling of rows to determine which columns and rows to load.
- Design the data model based on the above step.

#### Import libraries

In [1]:
import configparser
from pyspark.sql import types as T
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import shutil

#### Read config file

In [2]:
config = configparser.ConfigParser()
config.read('dl.cfg')

['dl.cfg']

#### Instantiate a Spark session

In [3]:
spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.5") \
        .getOrCreate()

#### Load and analyze I-94, airport, country, arrival mode, and visa type data from files

In [4]:
i94_data = config['LOCAL']['INPUT_DATA'] + "immigration_data_sample.csv"
i94_df = spark.read.options(header = 'true').csv(i94_data)
i94_df.printSchema()
print('I-94 rows loaded from file:', i94_df.count())
i94_df.show(5)

root
 |-- _c0: string (nullable = true)
 |-- cicid: string (nullable = true)
 |-- i94yr: string (nullable = true)
 |-- i94mon: string (nullable = true)
 |-- i94cit: string (nullable = true)
 |-- i94res: string (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: string (nullable = true)
 |-- i94mode: string (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: string (nullable = true)
 |-- i94bir: string (nullable = true)
 |-- i94visa: string (nullable = true)
 |-- count: string (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: string (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = tru

In [5]:
airport_data = config['LOCAL']['INPUT_DATA'] + "airport-codes_csv.csv"
airport_df = spark.read.options(header = 'true').csv(airport_data)
airport_df.printSchema()
print('Airport rows loaded from file:', airport_df.count())
airport_df.show(5)

root
 |-- ident: string (nullable = true)
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- elevation_ft: string (nullable = true)
 |-- continent: string (nullable = true)
 |-- iso_country: string (nullable = true)
 |-- iso_region: string (nullable = true)
 |-- municipality: string (nullable = true)
 |-- gps_code: string (nullable = true)
 |-- iata_code: string (nullable = true)
 |-- local_code: string (nullable = true)
 |-- coordinates: string (nullable = true)

Airport rows loaded from file: 55075
+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+
|ident|         type|                name|elevation_ft|continent|iso_country|iso_region|municipality|gps_code|iata_code|local_code|         coordinates|
+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+
|  00A|  

In [6]:
country_data = config['LOCAL']['INPUT_DATA'] + "i94_countries.json"
country_df = spark.read.option('multiline', 'true').json(country_data)
country_df.printSchema()
print('Country rows loaded from file:', country_df.count())
country_df.show(5)

root
 |-- code: long (nullable = true)
 |-- country: string (nullable = true)

Country rows loaded from file: 289
+----+--------------------+
|code|             country|
+----+--------------------+
| 582|MEXICO Air Sea, a...|
| 236|         AFGHANISTAN|
| 101|             ALBANIA|
| 316|             ALGERIA|
| 102|             ANDORRA|
+----+--------------------+
only showing top 5 rows



In [7]:
am_data = config['LOCAL']['INPUT_DATA'] + "i94_arrival_modes.json"
am_df = spark.read.option('multiline', 'true').json(am_data)
am_df.printSchema()
print('Arrival mode rows loaded from file:', am_df.count())
am_df.show()

root
 |-- mode: long (nullable = true)
 |-- name: string (nullable = true)

Arrival mode rows loaded from file: 4
+----+------------+
|mode|        name|
+----+------------+
|   1|         Air|
|   2|         Sea|
|   3|        Land|
|   9|Not reported|
+----+------------+



In [8]:
vt_data = config['LOCAL']['INPUT_DATA'] + "i94_visa_types.json"
vt_df = spark.read.option('multiline', 'true').json(vt_data)
vt_df.printSchema()
print('Visa type rows loaded from file:', vt_df.count())
vt_df.show()

root
 |-- code: long (nullable = true)
 |-- name: string (nullable = true)

Visa type rows loaded from file: 3
+----+--------+
|code|    name|
+----+--------+
|   1|Business|
|   2|Pleasure|
|   3| Student|
+----+--------+



***

### 2. Data Model and Structure

- The fact table will be derived from `data/immigration_data_sample.csv`.
- The airport dim table will be loaded from `data/airport-codes_csv.csv`.
- The country, arrival mode and visa type dim data was derived from `data/I94_SAS_Labels_Descriptions.SAS`.
- The country dim table will be loaded from `data/i94_countries.json`.
- The arrival mode dim table will be loaded from `data/i94_arrival_modes.json`.
- The visa type dim table will be loaded from `data/i94_visa_types.json`.

<img src='images/Data Model.png'>

| Table | Field | Data Type | Relationship | Comments |
| --- | --- | --- | --- | --- |
| **fact_i94** | cicid | int | | Unique ID from source data. |
| | port_of_entry | text | dim_airport.code (if arrival mode by air) | Port of entry. |
| | citizenship_country_code | int | dim_country.code | Immigrant's country of citizenship. |
| | residence_country_code | int | dim_country.code | Immigrant's country of residence. |
| | arrival_mode | int | dim_arrival_mode.mode | Immigrant's mode of arrival - by air, sea or land.|
| | visa_type | int | dim_visa_type.code | Immigrant's visa type - business, pleasure or student. |
| | year | int | | I-94 year. |
| | month | int | |  I-94 month. |
| | arrival_date | date | | Immigrant's arrival date. |
| | departure_date | date | | Immigrant's departure date. |
| | us_address_state | text | | Immigrant's address during visit. |
| | age | int | | Immigrant's age. |
| | gender | text | | Immigrant's gender. |
| **dim_airport** | iata_code | text | fact_i94.airport_code |   |
| | name | text | |   |
| | type | text | |   |
| | city | text | |   |
| | state | text | |   |
| **dim_country** | code | int | fact_i94.citizenship_country_code, fact_i94.residence_country_code |   |
| | country | text | |   |
| **dim_arrival_mode** | mode | int | fact_i94.arrival_mode |   |
| | name | text | |   |
| **dim_visa_type** | code | int | fact_i94.visa_type |   |
| | name | text | |   |


***

### 3. ETL Pipeline

<img src='images/Pipeline.png'>

#### Clear previously created data warehouse parquet tables

In [9]:
shutil.rmtree("spark-warehouse/")

#### Initialize variables for data quality checks later

In [10]:
tables = ['I-94', 'Airport', 'Country', 'Arrival Mode', 'Visa Type']

In [11]:
expected_row_count = {
    'I-94':0,
    'Airport':0,
    'Country':0,
    'Arrival Mode':0,
    'Visa Type':0
}

In [12]:
parquet_tables = {
    'I-94':'fact_i94.parquet',
    'Airport':'dim_airport.parquet',
    'Country':'dim_country.parquet',
    'Arrival Mode':'dim_arrival_mode.parquet',
    'Visa Type':'dim_visa_type.parquet'
}

#### Load I-94 data

1. Load I-94 data from CSV file.
2. Stage only needed columns.
3. Convert columns to appropriate data type.

In [13]:
drop_columns = ['_c0', 'count', 'dtadfile', 'visapost', 'occup', 'entdepa', 'entdepd', 'entdepu', 'matflag', 'biryear', 'dtaddto', 'insnum', 'airline', 'admnum', 'fltno', 'visatype']
i94_df = i94_df.select([column for column in i94_df.columns if column not in drop_columns])

In [14]:
int_columns = ['cicid', 'i94yr', 'i94mon', 'i94cit', 'i94res', 'arrdate', 'i94mode', 'depdate', 'i94bir', 'i94visa']
for column in int_columns:
    i94_df = i94_df.withColumn(column, i94_df[column].cast(T.IntegerType()))

In [15]:
i94_df.show(5)

+-------+-----+------+------+------+-------+-------+-------+-------+-------+------+-------+------+
|  cicid|i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|gender|
+-------+-----+------+------+------+-------+-------+-------+-------+-------+------+-------+------+
|4084316| 2016|     4|   209|   209|    HHW|  20566|      1|     HI|  20573|    61|      2|     F|
|4422636| 2016|     4|   582|   582|    MCA|  20567|      1|     TX|  20568|    26|      2|     M|
|1195600| 2016|     4|   148|   112|    OGG|  20551|      1|     FL|  20571|    76|      2|     M|
|5291768| 2016|     4|   297|   297|    LOS|  20572|      1|     CA|  20581|    25|      2|     M|
| 985523| 2016|     4|   111|   111|    CHM|  20550|      3|     NY|  20553|    19|      2|     F|
+-------+-----+------+------+------+-------+-------+-------+-------+-------+------+-------+------+
only showing top 5 rows



In [16]:
i94_df.createOrReplaceTempView("stg_i94")

#### Confirming that cicid's are unique in the data set

In [17]:
spark.sql("""
    select
        count(distinct cicid)
    from
        stg_i94
""").show()

+---------------------+
|count(DISTINCT cicid)|
+---------------------+
|                 1000|
+---------------------+



#### Write I-94 fact table

In [18]:
fact_i94_table = spark.sql("""
    select
        cicid,
        i94port as port_of_entry,
        i94cit as citizenship_country_code,
        i94res as residence_country_code,
        i94mode as arrival_mode,
        i94visa as visa_type,
        i94yr as year,
        i94mon as month,
        date_add(cast('1960-01-01' as date), arrdate) as arrival_date,
        date_add(cast('1960-01-01' as date), depdate) as departure_date,
        i94addr as us_address_state,
        i94bir as age,
        gender
    from
        stg_i94
""")

In [19]:
expected_row_count['I-94'] = fact_i94_table.count()

In [20]:
fact_i94_table.show(5)

+-------+-------------+------------------------+----------------------+------------+---------+----+-----+------------+--------------+----------------+---+------+
|  cicid|port_of_entry|citizenship_country_code|residence_country_code|arrival_mode|visa_type|year|month|arrival_date|departure_date|us_address_state|age|gender|
+-------+-------------+------------------------+----------------------+------------+---------+----+-----+------------+--------------+----------------+---+------+
|4084316|          HHW|                     209|                   209|           1|        2|2016|    4|  2016-04-22|    2016-04-29|              HI| 61|     F|
|4422636|          MCA|                     582|                   582|           1|        2|2016|    4|  2016-04-23|    2016-04-24|              TX| 26|     M|
|1195600|          OGG|                     148|                   112|           1|        2|2016|    4|  2016-04-07|    2016-04-27|              FL| 76|     M|
|5291768|          LOS|     

In [21]:
try:
    fact_i94_table.write.partitionBy('year', 'month').parquet(config['LOCAL']['OUTPUT_DATA'] + "fact_i94.parquet")
except Exception as e:
    print("Error:", e)

#### Load airport data

1. Load airport data from CSV file.
2. Stage only operational US airports with IATA code.
3. Stage only needed columns.

In [22]:
airport_df = airport_df.filter(airport_df.type != 'closed') \
                .filter(airport_df.iso_country == 'US') \
                .filter(airport_df.iata_code.isNotNull())

In [23]:
print('Operational airports in US with IATA code:', airport_df.count())

Operational airports in US with IATA code: 1956


In [24]:
drop_columns = ['ident', 'elevation_ft', 'continent', 'iso_country', 'gps_code', 'local_code', 'coordinates']
airport_df = airport_df.select([column for column in airport_df.columns if column not in drop_columns])

In [25]:
airport_df.show(5)

+-------------+--------------------+----------+-------------+---------+
|         type|                name|iso_region| municipality|iata_code|
+-------------+--------------------+----------+-------------+---------+
|small_airport|Ocean Reef Club A...|     US-FL|    Key Largo|      OCA|
|small_airport|Pilot Station Air...|     US-AK|Pilot Station|      PQS|
|small_airport|Crested Butte Air...|     US-CO|Crested Butte|      CSE|
|small_airport|   LBJ Ranch Airport|     US-TX| Johnson City|      JCY|
|small_airport|Metropolitan Airport|     US-MA|       Palmer|      PMX|
+-------------+--------------------+----------+-------------+---------+
only showing top 5 rows



In [26]:
airport_df.createOrReplaceTempView("stg_airport")

#### Confirming that IATA codes are unique in the data set

In [27]:
spark.sql("""
    select
        count(distinct iata_code)
    from
        stg_airport
""").show()

+-------------------------+
|count(DISTINCT iata_code)|
+-------------------------+
|                     1956|
+-------------------------+



#### Write airport dim table

In [28]:
dim_airport_table = spark.sql("""
    select
        iata_code,
        name,
        type,
        municipality as city,
        right(iso_region, 2) as state
    from
        stg_airport
""")

In [29]:
expected_row_count['Airport'] = dim_airport_table.count()

In [30]:
dim_airport_table.show(5)

+---------+--------------------+-------------+-------------+-----+
|iata_code|                name|         type|         city|state|
+---------+--------------------+-------------+-------------+-----+
|      OCA|Ocean Reef Club A...|small_airport|    Key Largo|   FL|
|      PQS|Pilot Station Air...|small_airport|Pilot Station|   AK|
|      CSE|Crested Butte Air...|small_airport|Crested Butte|   CO|
|      JCY|   LBJ Ranch Airport|small_airport| Johnson City|   TX|
|      PMX|Metropolitan Airport|small_airport|       Palmer|   MA|
+---------+--------------------+-------------+-------------+-----+
only showing top 5 rows



In [31]:
try:
    dim_airport_table.write.partitionBy('state').parquet(config['LOCAL']['OUTPUT_DATA'] + "dim_airport.parquet")
except Exception as e:
    print("Error:", e)

#### Load I-94 lookup data such as countries, arrival modes, and visa types

1. Lookup data for countries, arrival modes and visa types was created from the I-94 data dictionary.
2. This data in JSON format is used to create dimensions that provide meaning to codified information in I-94 fact table.

#### Countries

In [32]:
country_df.createOrReplaceTempView("stg_country")

#### Confirming that country codes are unique in the data set

In [33]:
spark.sql("""
    select
        count(distinct code)
    from
        stg_country
""").show()

+--------------------+
|count(DISTINCT code)|
+--------------------+
|                 289|
+--------------------+



#### Write country dim table

In [34]:
dim_country_table = spark.sql("""
    select
        code,
        country
    from
        stg_country
""")

In [35]:
expected_row_count['Country'] = dim_country_table.count()

In [36]:
dim_country_table.show(5)

+----+--------------------+
|code|             country|
+----+--------------------+
| 582|MEXICO Air Sea, a...|
| 236|         AFGHANISTAN|
| 101|             ALBANIA|
| 316|             ALGERIA|
| 102|             ANDORRA|
+----+--------------------+
only showing top 5 rows



In [37]:
try:
    dim_country_table.write.parquet(config['LOCAL']['OUTPUT_DATA'] + "dim_country.parquet")
except Exception as e:
    print("Error:", e)

#### Arrival modes

In [38]:
am_df.createOrReplaceTempView("stg_arrival_mode")

#### Write arrival mode dim table

In [39]:
dim_arrival_mode_table = spark.sql("""
    select
        mode,
        name
    from
        stg_arrival_mode
""")

In [40]:
expected_row_count['Arrival Mode'] = dim_arrival_mode_table.count()

In [41]:
dim_arrival_mode_table.show()

+----+------------+
|mode|        name|
+----+------------+
|   1|         Air|
|   2|         Sea|
|   3|        Land|
|   9|Not reported|
+----+------------+



In [42]:
try:
    dim_arrival_mode_table.write.parquet(config['LOCAL']['OUTPUT_DATA'] + "dim_arrival_mode.parquet")
except Exception as e:
    print("Error:", e)

#### Visa types

In [43]:
vt_df.createOrReplaceTempView("stg_visa_type")

#### Write visa type dim table

In [44]:
dim_visa_type_table = spark.sql("""
    select
        code,
        name
    from
        stg_visa_type
""")

In [45]:
expected_row_count['Visa Type'] = dim_visa_type_table.count()

In [46]:
dim_visa_type_table.show()

+----+--------+
|code|    name|
+----+--------+
|   1|Business|
|   2|Pleasure|
|   3| Student|
+----+--------+



In [47]:
try:
    dim_visa_type_table.write.parquet(config['LOCAL']['OUTPUT_DATA'] + "dim_visa_type.parquet")
except Exception as e:
    print("Error:", e)

***

### 4. Data Quality Checks

1. Confirm that number of rows loaded in parquet tables are as expected.
2. Check if any dimensions in fact table are missing in dim tables.

#### Row count check

In [48]:
for table in tables:
    df = spark.read.parquet(config['LOCAL']['OUTPUT_DATA'] + parquet_tables[table])
    if expected_row_count[table] == df.count():
        print("All rows loaded in parquet table related to {}. Expected rows: {}. Loaded rows: {}." \
              .format(table, expected_row_count[table], df.count()))
    else:
        print("Missing rows in parquet table related to {}. Expected rows: {}. Loaded rows: {}." \
              .format(table, expected_row_count[table], df.count()))

All rows loaded in parquet table related to I-94. Expected rows: 1000. Loaded rows: 1000.
All rows loaded in parquet table related to Airport. Expected rows: 1956. Loaded rows: 1956.
All rows loaded in parquet table related to Country. Expected rows: 289. Loaded rows: 289.
All rows loaded in parquet table related to Arrival Mode. Expected rows: 4. Loaded rows: 4.
All rows loaded in parquet table related to Visa Type. Expected rows: 3. Loaded rows: 3.


#### Missing dimension checks

1. Check that dimensions in I-94 fact table have a corresponding value in dim tables.
2. Write missing dimension values to a parquet table for remedial action later.

In [50]:
dim_missing_table = spark.sql("""
    select distinct
        'Airport' as dim,
        i94port as value
    from
        stg_i94
    where
        i94mode = 1 and
        i94port not in
            (
                select
                    iata_code
                from
                    stg_airport
            )
    union
    
    select distinct
        'Country' as dim,
         i94cit as value
    from
        stg_i94
    where
        i94cit not in
            (
                select
                    code
                from
                    stg_country
            )
    union
    
    select distinct
        'Country' as dim,
         i94res as value
    from
        stg_i94
    where
        i94res not in
            (
                select
                    code
                from
                    stg_country
            )
    union
    
    select distinct
        'Arrival Mode' as dim,
        i94mode as value
    from
        stg_i94
    where
        i94mode not in
            (
                select
                    mode
                from
                    stg_arrival_mode
            )
    union
    
    select distinct
        'Visa Type' as dim,
        i94visa as value
    from
        stg_i94
    where
        i94visa not in
            (
                select
                    code
                from
                    stg_visa_type
            )
""")

In [51]:
dim_missing_table.show()

+-------+-----+
|    dim|value|
+-------+-----+
|Airport|  WPB|
|Airport|  PHI|
|Country|  746|
|Airport|  SNJ|
|Airport|  DLR|
|Airport|  NOL|
|Country|  252|
|Airport|  NCA|
|Airport|  MCA|
|Airport|  CHI|
|Airport|  SHA|
|Airport|  SFR|
|Airport|  LOS|
|Country|  718|
|Airport|  FTL|
|Airport|  AGA|
|Airport|  MIL|
|Country|  133|
|Airport|  MON|
|Airport|  SAI|
+-------+-----+
only showing top 20 rows



In [52]:
try:
    dim_missing_table.write.partitionBy('dim').parquet(config['LOCAL']['OUTPUT_DATA'] + "dim_missing.parquet")
except Exception as e:
    print("Error:", e)

***

### 5. Summary

- Apache Spark was the tool of choice because it is a cluster-computing framework that is built for big data processing.
- Simply by adding clusters to an Apache Spark setup, millions of rows of source data can be processed.
- The data warehouse tables are stored in the distributed and high-performing Parquet format, which allows for fast and simultaneous analytic processing by 100s of users.
- The entire ETL process can be contained in a Python file, and scheduled, monitored and logged with Apache Airflow on an hourly, daily or monthly basis as per business requirement.